In [2]:
import pandas as pd

In [3]:
df_nasa=pd.read_csv("../functions/df_nasa_export_with_update.csv")

In [4]:
df_nasa.head()

,Project Title,Project Description,desc_api,benefits_text,primaryTxCode,primaryTxTitle,startTrl,currentTrl,endTrl,start_year,start_month,end_year,end_month,lead_org_type,Project Last Updated
0,High TRL Rover Lidar,Design and build a LIDAR engineering test unit...,NASA's future planetary rover-based missions s...,The Space Qualified Rover Light Detection and ...,TX10.1.1,Sensing and Perception for Autonomous Systems,4.0,6.0,6.0,2020.0,10.0,2024.0,9.0,NASA Center,10/11/2024 12:45:42 PM
1,Standardizing a Data and Power System for GSFC...,There are three primary Tasks associated with ...,There are three primary Tasks associated with ...,"To our knowledge, an NO2 sonde has never been ...",TX08.3.4,Environment Sensors,3.0,7.0,7.0,2024.0,5.0,2024.0,9.0,NASA Center,10/10/2024 07:46:27 PM
2,Development of ACADIA-to-CCD Camera Platform,Develop a platform to adapt the Goddard-develo...,Develop a platform to adapt the Goddard-develo...,The project will benefit systems and instrumen...,TX08.1.1,Detectors and Focal Planes,2.0,4.0,4.0,2022.0,10.0,2024.0,9.0,NASA Center,10/10/2024 07:43:13 PM
3,Atom Interferometer Gravity Gradiometer Techno...,Cold atom interferometers enable measurements ...,Cold atom interferometers enable measurements ...,The AIGG laser system lacks a path toward spac...,TX08.1.5,Lasers,4.0,4.0,4.0,2022.0,10.0,2024.0,9.0,NASA Center,10/10/2024 06:59:35 PM
4,Spaceflight Compatible Optical Atomic Strontiu...,Optical atomic clocks are critical tools for N...,Optical atomic clocks are critical tools for N...,OASIC leverages high accuracy timing and exqui...,TX05,"Communications, Navigation, and Orbital Debris...",1.0,2.0,2.0,2023.0,10.0,2024.0,9.0,NASA Center,10/10/2024 06:57:32 PM


In [5]:
df_nasa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19631 entries, 0 to 19630
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Project Title         19631 non-null  object 
 1   Project Description   18442 non-null  object 
 2   desc_api              18226 non-null  object 
 3   benefits_text         15256 non-null  object 
 4   primaryTxCode         19342 non-null  object 
 5   primaryTxTitle        19342 non-null  object 
 6   startTrl              15305 non-null  float64
 7   currentTrl            15129 non-null  float64
 8   endTrl                15076 non-null  float64
 9   start_year            19342 non-null  float64
 10  start_month           19342 non-null  float64
 11  end_year              19342 non-null  float64
 12  end_month             19342 non-null  float64
 13  lead_org_type         19025 non-null  object 
 14  Project Last Updated  19631 non-null  object 
dtypes: float64(7), obje

In [6]:
import pandas as pd

df = df_nasa.copy()

# --- 1) Last Updated ---
df["date"] = pd.to_datetime(df["Project Last Updated"], errors="coerce")

# --- 2) end_date güvenli oluşturma ---
# NaN yıl/ay varsa doldur
df["end_year"] = df["end_year"].fillna(0).astype(int)
df["end_month"] = df["end_month"].fillna(0).astype(int)

df["end_date"] = pd.to_datetime(df["end_year"].astype(str) + "-" + df["end_month"].astype(str) + "-01", errors="coerce")
df["detailed_text"] = (df_nasa["benefits_text"].fillna("") +"\n\n" + df_nasa["desc_api"].fillna("")).str.strip()
# --- 3) TRL seç ---

# 3) Çıktı dataframe
df_nasa_corpus = pd.DataFrame()
df_nasa_corpus["title"]       = df["Project Title"].where(df["Project Title"].notna(), None)
df_nasa_corpus["abstract"]        = df["Project Description"].where(df["Project Description"].notna(), None)
df_nasa_corpus["detailed_text"]= df["detailed_text"].fillna("").where(df["detailed_text"].notna(), None)
df_nasa_corpus["year"]        = df["date"].dt.year
df_nasa_corpus["month"]       = df["date"].dt.month
df_nasa_corpus["source_type"] = "nasa_project"
df_nasa_corpus["tech_field"]  = df["primaryTxTitle"].where(df["primaryTxTitle"].notna(), None)
df_nasa_corpus["trl"] = df.apply(
    lambda row: row["currentTrl"] if row["date"] < row["end_date"] else row["endTrl"],
    axis=1
)

df_nasa_corpus

,title,abstract,detailed_text,year,month,source_type,tech_field,trl
0,High TRL Rover Lidar,Design and build a LIDAR engineering test unit...,The Space Qualified Rover Light Detection and ...,2024,10,nasa_project,Sensing and Perception for Autonomous Systems,6.0
1,Standardizing a Data and Power System for GSFC...,There are three primary Tasks associated with ...,"To our knowledge, an NO2 sonde has never been ...",2024,10,nasa_project,Environment Sensors,7.0
2,Development of ACADIA-to-CCD Camera Platform,Develop a platform to adapt the Goddard-develo...,The project will benefit systems and instrumen...,2024,10,nasa_project,Detectors and Focal Planes,4.0
3,Atom Interferometer Gravity Gradiometer Techno...,Cold atom interferometers enable measurements ...,The AIGG laser system lacks a path toward spac...,2024,10,nasa_project,Lasers,4.0
4,Spaceflight Compatible Optical Atomic Strontiu...,Optical atomic clocks are critical tools for N...,OASIC leverages high accuracy timing and exqui...,2024,10,nasa_project,"Communications, Navigation, and Orbital Debris...",2.0
...,...,...,...,...,...,...,...,...
19626,Ultra-Lightweight Self-Deployable SMP Nanocomp...,Space deployable and rigidizable structures th...,Space deployable and rigidizable structures th...,2018,10,nasa_project,Ground Analogs for Space and Surface Systems,NaN
19627,"Data Mining Tool for Fleet Management, Phase I",None,,2018,10,nasa_project,Radiation-Hardened Extreme-Environment Compone...,NaN
19628,Integrated Multi-Range Rendezvous Control Syst...,None,,2018,10,nasa_project,Radiation-Hardened Extreme-Environment Compone...,NaN
19629,Transparent Integration of SINDA/G into Patran...,None,,2018,10,nasa_project,Radiation-Hardened Extreme-Environment Compone...,NaN


In [7]:
out_path = "../data/nasaproject_corpus.csv"
df_nasa_corpus.to_csv(out_path, index=False)